## 1. 데이터 로드

In [2]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_3_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_csv/"+'gcc6'+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_3_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_3_32.head()

(2329146, 2)
reset_index 완료
input data shape


,hex,bin
0,55,1
1,57,0
2,56,0
3,53,0
4,e8,0


## 2. Data 확인

In [3]:
# DATA 모양 확인
gcc6_3_32.values
print(gcc6_3_32.shape)

# 함수의 갯수
gcc6_3_32['bin'].value_counts()

# 함수 평균길이
gcc6_3_32['bin'].value_counts()[0]/gcc6_3_32['bin'].value_counts()[1]

array([['55', 1],
       ['57', 0],
       ['56', 0],
       ...,
       ['1c', 0],
       ['24', 0],
       ['c3', 0]], dtype=object)

(2329146, 2)


0    2317594
1      11552
Name: bin, dtype: int64

200.6227493074792

## 3. 함수의 갯수, 바이트 갯수의 빈공간 np.array 생성 

In [4]:
x = np.full((gcc6_3_32['bin'].value_counts()[1],80000), None, dtype='object')
x.shape
x.dtype

(11552, 80000)

dtype('O')

## 4. 함수 빈공간에 함수별로 한줄씩 값 대입

In [5]:
data = gcc6_3_32.values

row = -1
col = 0
col_list=[]

# data => [55,1], [89,1], [e5,1] ....
for i in data:
    # i[0] = 55, 89, e5... /  i[1] = 1, 0, 0, ....
    if i[1]==1:
        col_list.append(col) # col_list의 각 함수 길이들 넣는다.
        
        row+=1
        col=0
        x[row][col] = i[0]
#        print(row,col, end='/') # 오류났을 때 풀어보면 몇번째 행에서 값을 초과하는지 알 수 있음.
    else:
        x[row][col+1] = i[0]
        #if row >=1385:
        #    print(row,col, end=' ')
        col+=1
    
print('대입 완료')    

대입 완료


In [6]:
max(col_list)

75235

## 5. 최적의 x 찾기

In [7]:
# 예시
print('예시')
x[:, :6]
#print(len(col_list),'\n')

# 최소 3gram ~ 7gram 까지확인
for j in range(2, 7):
    tuple_ls = []
    for k in x[:, :j]:
        tuple_ls.append(tuple(k))

    set_ls = set(tuple_ls)
    print(j, 'gram일 때 ', len(set_ls),'쌍')

예시


array([['55', '57', '56', '53', 'e8', 'fc'],
       ['8b', '1c', '24', 'c3', None, None],
       ['55', '57', '89', 'd5', '56', '53'],
       ...,
       ['8b', '1c', '24', 'c3', None, None],
       ['55', '57', '56', '53', 'e8', 'fc'],
       ['8b', '1c', '24', 'c3', None, None]], dtype=object)

2 gram일 때  405 쌍
3 gram일 때  831 쌍
4 gram일 때  1270 쌍
5 gram일 때  1647 쌍
6 gram일 때  1906 쌍


In [8]:
# 임의 테스트
tls = []
for t in x[:, :4]:
    tls.append(tuple(t))
    
stls = set(tls)
stls

{('8d', '80', '30', '01'),
 ('8d', '70', '03', '89'),
 ('81', '7c', '24', '60'),
 ('8b', '45', 'cc', '8b'),
 ('c7', '44', '24', '40'),
 ('55', '31', 'c0', '89'),
 ('6a', '01', '8b', '44'),
 ('8b', '9d', '7c', 'ff'),
 ('b8', '0c', '00', '00'),
 ('8b', '4f', '20', '8b'),
 ('88', '41', '02', '0f'),
 ('bf', '02', '00', '00'),
 ('55', '57', 'b9', '03'),
 ('83', 'fe', '1f', '77'),
 ('be', '0c', '00', '00'),
 ('31', 'c0', 'e9', '7f'),
 ('8b', '32', '85', 'f6'),
 ('8d', '46', 'e4', '83'),
 ('8b', '45', 'fc', '89'),
 ('be', '83', '01', '00'),
 ('31', 'ed', '3b', '3c'),
 ('83', '7d', '00', '04'),
 ('83', 'a3', '00', '00'),
 ('0f', 'be', '55', '01'),
 ('be', '04', '00', '00'),
 ('83', '7c', '24', '38'),
 ('c7', '83', '8c', '00'),
 ('c7', '47', '40', '04'),
 ('8b', '7e', '10', '8b'),
 ('8b', '70', '14', '31'),
 ('31', 'f6', 'eb', 'df'),
 ('8b', '85', '70', '00'),
 ('8d', 'ab', '00', '00'),
 ('83', 'ec', '0c', '52'),
 ('8b', '53', '0c', '85'),
 ('8b', '45', '10', 'e8'),
 ('b8', '02', '01', '00'),
 